In [1]:
import math
import os
import torch
import torchani
import ase
import numpy as np
from ase.build import molecule

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Rcr = 5.2000e+00
Rca = 3.5000e+00
EtaR = torch.tensor([1.6000000e+01], device=device)
ShfR = torch.tensor([9.0000000e-01, 1.1687500e+00, 1.4375000e+00, 1.7062500e+00, 1.9750000e+00, 2.2437500e+00, 2.5125000e+00, 2.7812500e+00, 3.0500000e+00, 3.3187500e+00, 3.5875000e+00, 3.8562500e+00, 4.1250000e+00, 4.3937500e+00, 4.6625000e+00, 4.9312500e+00], device=device)
Zeta = torch.tensor([3.2000000e+01], device=device)
ShfZ = torch.tensor([1.9634954e-01, 5.8904862e-01, 9.8174770e-01, 1.3744468e+00, 1.7671459e+00, 2.1598449e+00, 2.5525440e+00, 2.9452431e+00], device=device)
EtaA = torch.tensor([8.0000000e+00], device=device)
ShfA = torch.tensor([9.0000000e-01, 1.5500000e+00, 2.2000000e+00, 2.8500000e+00], device=device)
num_species = 4

# AEV computer
aev_computer = torchani.AEVComputer(Rcr, Rca, EtaR, ShfR, EtaA, Zeta, ShfA, ShfZ, num_species)

In [2]:
def spe_to_int(ipt):
    opt = []
    dic = 'HCNO'
    for s in ipt:
        for i, c in enumerate([1, 6, 7, 8]):
            if s == c:
                opt.append(i)
                # print(dic[i], end=' ')
    return np.array(opt)

### Example 1 (single molecule with ASE)

In [3]:
# create a methane molecule in ASE
coordinates = [[0.03192167, 0.00638559, 0.01301679],
[-0.83140486, 0.39370209, -0.26395324],
[-0.66518241, -0.84461308, 0.20759389],
[0.45554739, 0.54289633, 0.81170881],
[0.66091919, -0.16799635, -0.91037834]]

CH4 = ase.Atoms(['C', 'H', 'H', 'H', 'H'], positions=coordinates)

# get species tensor, coords
species = torch.tensor([spe_to_int(CH4.get_atomic_numbers())], device=device) 

coordinates = torch.tensor([CH4.get_positions()], requires_grad=True, device=device)

print(species)
print(coordinates)

# get the AEV
aev_result=aev_computer((species, coordinates), cell=None, pbc=None)

print('Species:',aev_result.species)
print('AEV:',aev_result.aevs)

print('AEV Length:',aev_computer.aev_length)
print('AEV Shape:',aev_result.aevs.size())

tensor([[1, 0, 0, 0, 0]], device='cuda:0')
tensor([[[ 0.0319,  0.0064,  0.0130],
         [-0.8314,  0.3937, -0.2640],
         [-0.6652, -0.8446,  0.2076],
         [ 0.4555,  0.5429,  0.8117],
         [ 0.6609, -0.1680, -0.9104]]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)
Species: tensor([[1, 0, 0, 0, 0]], device='cuda:0')
AEV: tensor([[[5.5878e-01, 7.4497e-01, 1.2186e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0193e-02, 1.3975e-01, 3.0345e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0179e-02, 1.3571e-01, 1.9645e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0318e-05, 2.8428e-03, 8.7990e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.0563e-06, 1.5859e-03, 7.0698e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]]], device='cuda:0', dtype=torch.float64,
       grad_fn=<CatBackward>)
AEV Length: 384
AEV Shape: torch.Size([1, 5, 384])


### Example 2 (single molecule load directly)

In [4]:
coordinates = torch.tensor(
    [[[0.03192167, 0.00638559, 0.01301679],
      [-0.83140486, 0.39370209, -0.26395324],
      [-0.66518241, -0.84461308, 0.20759389],
      [0.45554739, 0.54289633, 0.81170881],
      [0.66091919, -0.16799635, -0.91037834]]],
    requires_grad=True,
    device=device)

species = torch.tensor([[1, 0, 0, 0, 0]], device=device)

# get the AEV
aev_result=aev_computer((species, coordinates), cell=None, pbc=None)

print('Species:',aev_result.species)
print('AEV:',aev_result.aevs)

print('AEV Length:',aev_computer.aev_length)
print('AEV Shape:',aev_result.aevs.size())

Species: tensor([[1, 0, 0, 0, 0]], device='cuda:0')
AEV: tensor([[[5.5878e-01, 7.4497e-01, 1.2186e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0193e-02, 1.3975e-01, 3.0345e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0179e-02, 1.3571e-01, 1.9645e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0318e-05, 2.8428e-03, 8.7991e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.0563e-06, 1.5859e-03, 7.0698e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]]], device='cuda:0', grad_fn=<CatBackward>)
AEV Length: 384
AEV Shape: torch.Size([1, 5, 384])


### Example 3 (multiple molecule with ASE)

In [5]:
def pad_ase_molecule(mol_list, device):
    """
    input: ase molecule list
    return: [species, coordinates]
            padded species and coordinates input for torchani (in torch tensor)
    """
    species = []
    coordinates = []
    for mol in mol_list:
        species.append(torch.from_numpy(spe_to_int(mol.get_atomic_numbers())).to(device))
        coordinates.append(torch.from_numpy(mol.get_positions()).to(device))
    # before padding
    print("before padding")
    print(species)
    print(coordinates)
    # after padding
    print("after padding")
    species = torch.nn.utils.rnn.pad_sequence(species,
                                              batch_first=True,
                                              padding_value=-1,)
    coordinates = torch.nn.utils.rnn.pad_sequence(coordinates,
                                              batch_first=True,
                                              padding_value=0.0, )

    print(species)
    print(coordinates)
    return species, coordinates

In [6]:
NH3 = molecule('NH3')
CH4 = molecule('CH4')
print(NH3.get_atomic_numbers())
print(CH4.get_atomic_numbers())

[7 1 1 1]
[6 1 1 1 1]


In [7]:
# species are padded with -1
# coordinates are padded with 0
species, coordinates = pad_ase_molecule([NH3, CH4], device)

before padding
[tensor([2, 0, 0, 0], device='cuda:0'), tensor([1, 0, 0, 0, 0], device='cuda:0')]
[tensor([[ 0.0000,  0.0000,  0.1165],
        [ 0.0000,  0.9397, -0.2718],
        [ 0.8138, -0.4699, -0.2718],
        [-0.8138, -0.4699, -0.2718]], device='cuda:0', dtype=torch.float64), tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.6291,  0.6291,  0.6291],
        [-0.6291, -0.6291,  0.6291],
        [ 0.6291, -0.6291, -0.6291],
        [-0.6291,  0.6291, -0.6291]], device='cuda:0', dtype=torch.float64)]
after padding
tensor([[ 2,  0,  0,  0, -1],
        [ 1,  0,  0,  0,  0]], device='cuda:0')
tensor([[[ 0.0000,  0.0000,  0.1165],
         [ 0.0000,  0.9397, -0.2718],
         [ 0.8138, -0.4699, -0.2718],
         [-0.8138, -0.4699, -0.2718],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.6291,  0.6291,  0.6291],
         [-0.6291, -0.6291,  0.6291],
         [ 0.6291, -0.6291, -0.6291],
         [-0.6291,  0.6291, -0.6291]]], device='cuda:

In [8]:
# get the AEV
aev_result=aev_computer((species, coordinates), cell=None, pbc=None)

print('Species:',aev_result.species)
print('AEV:',aev_result.aevs)

print('AEV Length:',aev_computer.aev_length)
print('AEV Shape:',aev_result.aevs.size())

Species: tensor([[ 2,  0,  0,  0, -1],
        [ 1,  0,  0,  0,  0]], device='cuda:0')
AEV: tensor([[[5.4783e-01, 4.7095e-01, 4.0137e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.1316e-05, 1.3368e-02, 2.1786e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.1315e-05, 1.3368e-02, 2.1786e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.1315e-05, 1.3368e-02, 2.1786e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[5.0362e-01, 8.1023e-01, 1.2923e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [2.3394e-06, 1.4183e-03, 8.5248e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [2.3394e-06, 1.4183e-03, 8.5248e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [2.3394e-06, 1.4183e-03, 8.5248e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [2.339

### Example 4 (multiple molecule load directly)

In [9]:
coordinates = torch.tensor(
    [[[0.03192167, 0.00638559, 0.01301679],
      [-0.83140486, 0.39370209, -0.26395324],
      [-0.66518241, -0.84461308, 0.20759389],
      [0.45554739, 0.54289633, 0.81170881],
      [0.66091919, -0.16799635, -0.91037834]],
     [[-4.1862600, 0.0575700, -0.0381200], 
      [-3.1689400, 0.0523700, 0.0200000],
      [-4.4978600, 0.8211300, 0.5604100], 
      [-4.4978700, -0.8000100, 0.4155600],
      [0.00000000, -0.00000000, -0.00000000]]],
    requires_grad=True,
    device=device)

species = torch.tensor([[1, 0, 0, 0, 0], [2, 0, 0, 0, -1]], device=device)

# get the AEV
aev_result=aev_computer((species, coordinates), cell=None, pbc=None)

print('Species:',aev_result.species)
print('AEV:',aev_result.aevs)

print('AEV Length:',aev_computer.aev_length)
print('AEV Shape:',aev_result.aevs.size())

Species: tensor([[ 1,  0,  0,  0,  0],
        [ 2,  0,  0,  0, -1]], device='cuda:0')
AEV: tensor([[[5.5878e-01, 7.4497e-01, 1.2186e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0193e-02, 1.3975e-01, 3.0345e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0179e-02, 1.3571e-01, 1.9645e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0318e-05, 2.8428e-03, 8.7991e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [4.0563e-06, 1.5859e-03, 7.0698e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[5.4306e-01, 4.7579e-01, 4.1326e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.1441e-05, 1.3381e-02, 2.1795e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.1441e-05, 1.3381e-02, 2.1795e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.1437e-05, 1.3381e-02, 2.1795e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.000